### Install required package

In [ ]:
!pip install -q torch torchtext pandas transformers tqdm sentencepiece datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
pip install --upgrade pyarrow

### Import all Packages

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from tqdm import tqdm
import time
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import ast
#import datasets
#from datasets import load_dataset, DatasetDict, Dataset
import time

### Load Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#data_sample=load_dataset(r"load dataset from hugging face")

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Fine_Tunning/CyberSecurityDataset.csv")

In [ ]:
# # If you have already two columns not use this line
# updated_data=[{"Question":item["Question"],"Answer":item["Answer"]} for item in data_sample["train"]]

In [ ]:
# # Convert into dataframe
# df=pd.DataFrame(updated_data)

In [ ]:
df=df[["Question","Answer"]]

In [ ]:
df.head()

,Question,Answer
0,What is a firewall in cybersecurity?,A firewall is a network security device that m...
1,What are the common types of cyberattacks?,"Common types of cyberattacks include phishing,..."
2,How does encryption help in cybersecurity?,Encryption helps protect sensitive data by con...
3,What is two-factor authentication (2FA)?,Two-factor authentication (2FA) is a security ...
4,What is the difference between a virus and a w...,"A virus needs a host program to spread, wherea..."


In [ ]:
df.count()

,0
Question,916
Answer,916


In [ ]:
if torch.cuda.is_available():
  device=torch.device("cuda")
else:
  device=torch.device("cpu")

In [ ]:
# Model params
BATCH_SIZE = 8
tokenizer = GPT2Tokenizer.from_pretrained('distilbert/distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilbert/distilgpt2').to(device)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Dataset Class
class LanguageDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.labels = df.columns
        self.data = df.to_dict(orient='records')
        self.tokenizer = tokenizer
        self.max_length = self.fittest_max_length(df)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx][self.labels[0]]  # Question
        y = self.data[idx][self.labels[1]]  # Answer
        text = f"Question: {x} | Answer: {y}"
        tokens = self.tokenizer.encode_plus(text, return_tensors='pt', max_length=128, padding='max_length', truncation=True)
        return tokens
    def fittest_max_length(self, df):
        # Convert values to strings, ignore NaNs
        max_len_question = max(len(str(val)) for val in df[self.labels[0]].dropna())
        max_len_answer = max(len(str(val)) for val in df[self.labels[1]].dropna())

        max_length = max(max_len_question, max_len_answer)

        # Find the nearest power of 2 greater than max_length
        x = 2
        while x < max_length:
            x *= 2
        return x

In [ ]:
# Create dataset instance
data_sample = LanguageDataset(df, tokenizer)

# Split data into training and validation sets
train_size = int(0.8 * len(data_sample))
valid_size = len(data_sample) - train_size
train_data, valid_data = random_split(data_sample, [train_size, valid_size])

# Data loaders
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=BATCH_SIZE)

In [ ]:
# Training loop
num_epochs = 10
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=5e-4)

results = pd.DataFrame(columns=['epoch', 'transformer', 'batch_size', 'gpu',
                                'training_loss', 'validation_loss', 'epoch_duration_sec'])

for epoch in range(num_epochs):
    start_time = time.time()

    # Training
    model.train()
    epoch_training_loss = 0
    train_iterator = tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}")
    for batch in train_iterator:
        optimizer.zero_grad()
        inputs = batch['input_ids'].squeeze(1).to(device)
        targets = inputs.clone()
        outputs = model(input_ids=inputs, labels=targets)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        epoch_training_loss += loss.item()

    avg_epoch_training_loss = epoch_training_loss / len(train_iterator)

    # Validation
    model.eval()
    epoch_validation_loss = 0
    valid_iterator = tqdm(valid_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}")
    with torch.no_grad():
        for batch in valid_iterator:
            inputs = batch['input_ids'].squeeze(1).to(device)
            targets = inputs.clone()
            outputs = model(input_ids=inputs, labels=targets)
            loss = outputs.loss
            epoch_validation_loss += loss.item()

    avg_epoch_validation_loss = epoch_validation_loss / len(valid_loader)
    end_time = time.time()
    epoch_duration_sec = end_time - start_time

    new_row = {'transformer': 'distilgpt2', 'batch_size': BATCH_SIZE, 'gpu': 0, 'epoch': epoch+1,
               'training_loss': avg_epoch_training_loss, 'validation_loss': avg_epoch_validation_loss,
               'epoch_duration_sec': epoch_duration_sec}
    results.loc[len(results)] = new_row

    print(f"Epoch: {epoch+1}, Validation Loss: {avg_epoch_validation_loss}")

Validation Epoch 1/10: 100%|██████████| 24/24 [01:29<00:00,  3.73s/it]


Epoch: 1, Validation Loss: 0.9004273551205794


Validation Epoch 2/10: 100%|██████████| 24/24 [01:27<00:00,  3.65s/it]


Epoch: 2, Validation Loss: 0.8943575719992319


Training Epoch 3/10:  97%|█████████▋| 90/93 [18:15<00:36, 12.22s/it]

In [ ]:
# Generation Test
input_str = "What is phishing?"  # Define the input prompt for the model

# Encode the input string into tensor format for the model
input_ids = tokenizer.encode(input_str, return_tensors='pt').to(device)

# Generate responses based on the input tensor
output = model.generate(
    input_ids,                      # The tensor representation of the input text
    max_length=200,                 # Maximum length of the generated sequence (in tokens)
    num_return_sequences=2,        # Number of different sequences to generate
    do_sample=True,                 # Enable sampling (as opposed to greedy decoding)
    top_k=8,                        # The number of highest probability tokens to keep for top-k sampling
    top_p=0.95,                     # Cumulative probability for nucleus sampling (top-p sampling)
    temperature=0.5,                # Controls the randomness of predictions by scaling logits before sampling
    repetition_penalty=1.2          # Penalizes repeated tokens to reduce repetitiveness
)

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded_output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is phishing? | Answer: A social engineering scam where attackers deceive people into revealing sensitive information or installing malware. Attackers often impersonate trusted entities and create a sense of urgency, such as threatening to close a victim's bank account. Another tactic is using fake news articles to trick victims into clicking malicious links, which lead them to believe that legitimate emails may be sent by fake websites or fake websites.
